In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wdtools import *

In [17]:
sets = list(range(1,9))

In [10]:
data = get_all_wd(sets = list(range(1,9)))

In [11]:
data.shape

(5861, 26)

In [12]:
data.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [19]:
df = data.groupby('SetID', as_index=False)['wetdet_delin_number'].nunique()

In [20]:
df.rename(columns={'wetdet_delin_number': 'total'}, inplace=True)

In [28]:
def read_remapping(outnm):
    file = fr'{outpath}\to_review\re_mapping{outnm}.txt' 
    with open(file) as f: 
        remapIDs = f.readlines()
    dt = remapIDs[0].split(', ')
    return dt

In [52]:
qaqc = read_remapping(outnm='')
qaqc_set7 = read_remapping(outnm='_Set007')
qaqc_set8 = read_remapping(outnm='_Set008')

In [55]:
qaqc_list = qaqc + qaqc_set7 + qaqc_set8

In [40]:
len(qaqc + qaqc_set7 + qaqc_set8)

342

In [42]:
def read_issues(setID):
    issues = pd.read_excel(fr'{outpath}\to_review\{setID}_Mapping_Issues.xlsx')
    issueIDs = list(issues.wetdet_delin_number.unique())
    return issueIDs

In [54]:
issues6 = read_issues(setID = 'Set006')
issues7 = read_issues(setID = 'Set007')
issues8 = read_issues(setID = 'Set008')

In [43]:
def read_unmatched(setID):
    unmatched = pd.read_csv(fr'{outpath}\to_review\unmatched_df_{setID}_2.csv')
    unmatchedIDs = list(unmatched.wetdet_delin_number.unique())
    return unmatchedIDs

In [45]:
unmatched6 = read_unmatched(setID = 'Set006')
unmatched7 = read_unmatched(setID = 'Set007')
unmatched8 = read_unmatched(setID = 'Set008')

In [56]:
qaqc_list = qaqc_list + issues6 + issues7 + issues8 
qaqc_list = qaqc_list + unmatched6 + unmatched7 + unmatched8

In [47]:
len(qaqc_list)

434

In [57]:
qaqc_list = unique(qaqc_list)

In [58]:
len(qaqc_list)

412

In [51]:
data[data.wetdet_delin_number == 'WD2022-0112']

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,response_date,reissuance_response_date,project_id,site_id,record_ID,notes,recyear,IDyear,missinglot,SetID
1077,WD2022-0112,36S05W23D,1014,485 Avenue de Teresa,Grants Pass,Josephine,Tracy and Patrick Leenknecht,WD2022-0112 Offsite WetDet,42.424164,-123.256472,...,2022-03-30,NaN,83490.0,138619,1078,NaN,2022,2022,N,1


In [62]:
set_dict = data.set_index('wetdet_delin_number').to_dict(orient='dict')['SetID'] 

In [79]:
qaqc_df = count_lst_ele(set_dict, qaqc_list, colnm = 'SetID', ctnm = 'Unmapped')

In [80]:
qaqc_df.head()

,SetID,Unmapped
0,1,121
1,2,22
2,3,30
3,4,67
4,5,10


In [68]:
ndf = data.groupby('SetID', as_index=False)['wetdet_delin_number'].nunique()

In [69]:
ndf.rename(columns={'wetdet_delin_number': 'Total'}, inplace=True)

In [70]:
ndf.head()

,SetID,Total
0,1,1003
1,2,434
2,3,433
3,4,676
4,5,225


In [73]:
df = data.groupby('SetID', as_index=False)['county'].apply(lambda x: ', '.join(unique(x)))

In [75]:
df.rename(columns={'county': 'County'}, inplace=True)

In [81]:
out = df.merge(ndf, on='SetID', how='left')

In [82]:
out = out.merge(qaqc_df, on='SetID', how='left')

In [86]:
out.loc[:, 'Percent Completed'] = round((1-out.Unmapped/out.Total)*100, 2)

In [89]:
out.loc[:, 'No. County'] = out.County.apply(lambda x: len(x.split(', ')))

In [90]:
out.to_excel(fr"{inpath}\reporting\mapping_status.xlsx", index=False)  

In [115]:
def export_unmapped(sid):
    wdlist = data[(data.wetdet_delin_number.isin(qaqc_list)) & (data.SetID == sid)]
    wd_list = list(wdlist.wetdet_delin_number.unique())
    with open(fr'{inpath}\output\to_review\Unmapped_Set00{sid}.txt', 'w') as f:
        f.write(", ".join(wd_list))
    return wd_list

In [118]:
for sid in range(1, 9):
    out = export_unmapped(sid=sid)

In [124]:
def status_report(cat='SetID'):
    dict2check = data.set_index('wetdet_delin_number').to_dict(orient='dict')[cat]
    qaqc_df = count_lst_ele(dict2check, qaqc_list, colnm = cat, ctnm = 'Unmapped')
    ndf = data.groupby(cat, as_index=False)['wetdet_delin_number'].nunique()
    ndf.rename(columns={'wetdet_delin_number': 'Total'}, inplace=True)
    if cat == 'SetID':
        df = data.groupby(cat, as_index=False)['county'].apply(lambda x: ', '.join(unique(x)))
        df.rename(columns={'county': 'County'}, inplace=True)
        out = df.merge(ndf, on=cat, how='left')
        out = out.merge(qaqc_df, on=cat, how='left')
        out.loc[:, 'No. County'] = out.County.apply(lambda x: len(x.split(', ')))
    elif cat == 'county':
        out = ndf.merge(qaqc_df, on=cat, how='left')
        out.rename(columns={'county': 'County'}, inplace=True)
    out.loc[:, 'Percent Completed'] = round((1-out.Unmapped/out.Total)*100, 2)
    out.to_excel(fr"{inpath}\reporting\mapping_status_{cat}.xlsx", index=False)
    return out

In [125]:
out = status_report(cat='county')

In [15]:
ndf = data.groupby('county', as_index=False)['wetdet_delin_number'].nunique()

In [16]:
ndf.rename(columns={'wetdet_delin_number': 'total'}, inplace=True)

In [52]:
# qaqc combines partially matched and unmatched
# here, qaqc doesn't include the false positives in the matched records
qaqc = []
# unmatched
unmt = []
# wdID with questions
issue = []
# mapped
mapped = []
# reviewed
reviewed = []

In [53]:
qaqcdf = pd.DataFrame(columns=['SetID', 'Counties', 'Total Count', 'Total QAQC Count', 
                               'Partial and Mythical Count', 'Unmatched Count', 'Issue ID Count', 'Mapped Count'])

In [54]:
for setid in sets:
    print(setid)
    wd_dt = data[data.SetID==setid]
    counties = wd_dt['county'].unique()
    print(counties)
    N = data[data.SetID==setid]['wetdet_delin_number'].nunique()
    print(N)
    matched = gpd.read_file(os.path.join(inpath + '\\output\\matched\\', f'matched_records_{setid}_edited.shp'))
    final = gpd.read_file(os.path.join(inpath + '\\output\\final\\', f'mapped_wd_{setid}.shp'))
    a = list(matched[matched.notes.notnull()].wdID.unique())
    unmatched = pd.read_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setid}_2.csv'))
    b = list(unmatched.wetdet_delin_number.unique())
    unmatched_wd = [x for x in b if x not in a]
    unmatched3 = unmatched[unmatched.wetdet_delin_number.isin(unmatched_wd)]
    unmatched3.to_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setid}_3.csv'), index=False)
    unmt.append(unmatched_wd)
    qaqc.append(unique(a+b))
    issues = pd.read_csv(os.path.join(inpath, "output", "to_review", f"{setid}_Mapping_Issues.csv"))
    issueIDs = list(issues.wetdet_delin_number.unique())
    issue.append(issueIDs)
    c = list(final[final.code==1].wdID.unique())
    mapped.append(c)
    to_check = [wid for wid in unique(a+b) if wid not in (issueIDs+c)]
    if len(to_check) > 0:
        file = outpath+f'\\matched\\{setid}_reviewed.txt'
        if os.path.isfile(file):
            with open(file) as f:
                rev = f.readlines()
                d = rev[0].split(", ")
                to_check = [wid for wid in to_check if wid not in d]
                reviewed.append(d)
    print(f"""partial and mythical count - {len(a)}, 
          unmatched count - {len(unmatched_wd)},
          total QAQC count - {len(unique(a+b))},
          issue ID count -{len(issueIDs)},
          mapped count - {len(c)},
          total count - {N},
          to check - {to_check}""")
    qaqcdf = qaqcdf.append({'SetID': setid,
                            'Counties': ', '.join(list(counties)),
                            'Total Count': N,
                            'Total QAQC Count': len(unique(a+b)), 
                            'Partial and Mythical Count': len(a), 
                            'Unmatched Count': len(unmatched_wd),
                            'Issue ID Count': len(issueIDs),
                            'Mapped Count': len(c)}, ignore_index=True)

Set001
['Clatsop' 'Coos' 'Deschutes' 'Harney' 'Jackson' 'Josephine' 'Lake'
 'Marion']
1003
partial and mythical count - 264, 
          unmatched count - 14,
          total QAQC count - 278,
          issue ID count -4,
          mapped count - 277,
          total count - 1003,
          to check - []
Set002
['Baker' 'Benton' 'Clackamas' 'Morrow']
434
partial and mythical count - 138, 
          unmatched count - 14,
          total QAQC count - 152,
          issue ID count -15,
          mapped count - 141,
          total count - 434,
          to check - []
Set003
['Columbia' 'Grant' 'Klamath' 'Lincoln' 'Tillamook']
433
partial and mythical count - 127, 
          unmatched count - 11,
          total QAQC count - 138,
          issue ID count -9,
          mapped count - 125,
          total count - 433,
          to check - []
Set004
['Crook' 'Curry' 'Hood River' 'Jefferson' 'Linn' 'Multnomah' 'Wasco'
 'Yamhill' 'Yamhill and Washington']
676
partial and mythical count - 208, 
 

In [72]:
4+15+9+18

46

review_loop_r1(wdid_list=['WD2017-0331', 'WD2018-0678', 'WD2019-0074', 'WD2019-0202', 'WD2019-0208', 
                          'WD2019-0211', 'WD2019-0320', 'WD2019-0389', 'WD2019-0395', 
                          'WD2019-0582', 'WD2020-0207', 'WD2020-0336', 'WD2020-0568', 'WD2022-0214'], 
               partial=True, idx=True, wddf=data)

In [16]:
file=os.path.join(inpath, 'reporting', f'SetStatusReport{str(date.today()).replace("-", "")}.xlsx')

In [17]:
qaqcdf.to_excel(file, index=False)

In [18]:
reformat(file)

Removed format
Autofitted columns...


In [19]:
qaqc = unique(np.concatenate(qaqc))

In [20]:
len(qaqc)

788

In [23]:
df = data[['wetdet_delin_number', 'county']]

In [24]:
wdcnt_dict = df.set_index('wetdet_delin_number').to_dict(orient='dict')['county']

In [23]:
# count QAQC by county
qaqc_df = count_lst_ele(wdcnt_dict, qaqc, 'QAQC')

In [24]:
colnm='QAQC_comp'

In [56]:
comp_df = count_lst_ele(wdcnt_dict, flatten(mapped)+flatten(issue)+flatten(reviewed), colnm)

In [57]:
out = ndf.merge(qaqc_df, on='county', how='left')

In [58]:
out1 = out.merge(comp_df, on='county', how='left')

In [59]:
sel1 = out1.QAQC_comp.isnull()
out1.loc[sel1, colnm] = 0

set4cty = ['Crook', 'Curry', 'Hood River', 'Jefferson', 'Linn', 'Multnomah', 'Wasco', 
           'Yamhill', 'Yamhill and Washington']

In [60]:
#sel2 = (out1.QAQC_comp > out1.QAQC) | (~out1.county.isin(set4cty))
sel2 = out1.QAQC_comp > out1.QAQC

In [61]:
out1.loc[sel2, colnm] = out1.loc[sel2, 'QAQC']

In [62]:
out1[colnm] = out1[colnm].apply(np.int64)

In [63]:
out1['QAQC_pct'] = out1.apply(lambda row: str(round(row.QAQC/row.total*100,1))+('%'), axis=1)

In [64]:
out1['QAQC_comp_pct'] = out1.apply(lambda row: str(round(row.QAQC_comp/row.QAQC*100,1))+('%'), axis=1)

In [65]:
out1

,county,total,QAQC,QAQC_comp,QAQC_pct,QAQC_comp_pct
0,Baker,38,12,12,31.6%,100.0%
1,Benton,91,30,30,33.0%,100.0%
2,Clackamas,275,94,94,34.2%,100.0%
3,Clatsop,137,41,41,29.9%,100.0%
4,Columbia,93,33,33,35.5%,100.0%
5,Coos,120,27,27,22.5%,100.0%
6,Crook,35,11,11,31.4%,100.0%
7,Curry,17,3,3,17.6%,100.0%
8,Deschutes,172,38,38,22.1%,100.0%
9,Grant,23,6,6,26.1%,100.0%


In [66]:
out1.columns = ['County', 'Total Count', 'QAQC Count', 'QAQC Completed Count', 
                'QAQC Count Percentage', 'QAQC Completed Count Percentage']

In [67]:
file=os.path.join(inpath, 'reporting', f'StatusReport{str(date.today()).replace("-", "")}.xlsx')

In [68]:
out1.to_excel(file, index=False)

In [69]:
reformat(file)

Removed format
Autofitted columns...
